In [1]:
import os
import litellm
import base64
import json
import warnings
from dotenv import load_dotenv
import numpy as np
import matplotlib.pyplot as plt

# Load environment variables
load_dotenv()

# Suppress specific Pydantic warnings
warnings.filterwarnings("ignore", category=UserWarning, module="pydantic")

custom_api_base = "https://litellmproxy.osu-ai.org" 
astro1221_key = os.getenv("ASTRO1221_API_KEY")

conversation_prompt = "Have the characters Dr. Daniel Beer (an inquisitive and enthusiastic astrophysicist who studies cosmology, including dark matter and dark energy, galactic evolution and quasars, and the intergalactic medium (IGM)),  Dr. Davis Milk (a curious and passionate astrophysicist who studies supermassive black holes), and Dr. Phil Marg (an excited astrophysicist who studies cosmology, specifically galactic evolution and active galactic nuclei (AGN)) talk about star formation in character by connecting what they research to the topic of star formation."
conversation_history = [{"role": "user", "content": conversation_prompt}]

In [2]:
def prompt_llm(messages, model="openai/GPT-4.1-mini", temperature=0.5, max_tokens=1000, tools=None, verbose=True):
    """
    Send a prompt or conversation to an LLM using LiteLLM and return the response.
    """
    if isinstance(messages, str):
        messages = [{"role": "user", "content": conversation_prompt}] #changed "content": messages to --> "content":conversation_prompt (12/14)
    if not (isinstance(temperature, (int, float)) and 0 <= temperature <= 2): #kept all of this the same (12/14)
        raise ValueError("temperature must be a float between 0 and 2 (inclusive).")
    if not (isinstance(max_tokens, int) and max_tokens > 0):
        raise ValueError("max_tokens must be a positive integer.")
    #creates a prompt function prompt_llm(prompt, shows GPT version, med creativity, response length, tools, verbose = true provides detailed info)

    try: 
        print("Contacting LLM via University Server...")
        conversation_response = litellm.completion(
            model=model,
            messages=messages,
            tools=tools,
            api_base=custom_api_base,
            api_key=astro1221_key,
            temperature=temperature,
            max_tokens=max_tokens
        )
        conversation_answer = conversation_response['choices'][0]['message']['content'] #choices? what does this do, ask in class (12/12), added "conversation'" in front of response and answer variables
        if verbose: 
            print(f"\nSUCCESS! Here is the discussion from {model}:\n")
            print(conversation_answer + "\n")
            print(f"{conversation_answer['name']}: {conversation_answer['reply']}")
            conversation_history += f"{conversation_answer['name']}: {conversation_answer['reply']}\n"
    except Exception as e:
        print(f"\nERROR: Could not connect. Details:\n{e}")    
        conversation_response = None
    return conversation_response

In [3]:
prompt_llm(conversation_prompt)

Contacting LLM via University Server...

SUCCESS! Here is the discussion from openai/GPT-4.1-mini:

**Dr. Daniel Beer:**  
You know, when I think about star formation, I’m immediately drawn to the larger cosmological context. The intergalactic medium—the vast reservoirs of gas between galaxies—is essentially the raw material for star formation. The way dark matter scaffolds the universe influences how gas clouds collapse and form stars. Without the gravitational wells created by dark matter halos, baryonic matter wouldn’t clump enough to ignite new stars. So, star formation isn’t just a local process; it’s deeply tied to the cosmic web and the evolution of galaxies over billions of years.

**Dr. Davis Milk:**  
Absolutely, Daniel. And from my perspective studying supermassive black holes, star formation and black hole growth are often intertwined. In the dense central regions of galaxies, where supermassive black holes reside, the inflow of gas that fuels black hole accretion can also 